## Text classification

In [1]:
import pandas as pd
import numpy as np

In [34]:
#pip install -U imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 3.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 31.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Oversampling and under sampling
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from collections import Counter

In [3]:
combined_df = pd.read_csv("combined_profile.csv")

In [4]:
combined_df.head()

,username,age,l_occupation,status,gender,filtered sentence,y
0,williamgeorge,44,doctoral student,single,male,Tired Millennial outlook I see around Is woman...,0
1,lion10x,59,legal services,divorced,male,I good man healthy honest high educated always...,0
2,Luistroy,42,construction,single,male,Im 56 weigh 140 black hair brown eyes slim loo...,0
3,Jonjon46,49,landscaper/cook an baker jack of all trades,single,male,Im laid back easy going guy funny love cars mu...,0
4,vitor78,43,businessman,divorced,male,I simple person really loves live I nature lov...,0


In [9]:
# features
X = combined_df['filtered sentence']

# target
y = combined_df['y']

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 424)

In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100000)
tfidf_vect.fit(X)
xtrain_tfidf =  tfidf_vect.transform(X_train)
xvalid_tfidf =  tfidf_vect.transform(X_test)

# Naive Bayes classifier
- Taken from: https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a

In [11]:
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#WITHOUT OVERSAMPLING/UNDER SAMPLING
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])
text_clf = text_clf.fit(X_train, y_train)

In [25]:
predictions = text_clf.predict(X_test)

### Evaluating the NB classifier

In [26]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("original dataset")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

original dataset
Accuracy: 0.8718428437792329
Precision: 0.7211238293444329
recall: 0.9914163090128756
f1_score: 0.8349397590361446


In [56]:
#random oversampling
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100000)
tfidf_vect.fit(X)
xtrain_tfidf =  tfidf_vect.transform(X_train)
xvalid_tfidf =  tfidf_vect.transform(X_test)

text_clf = MultinomialNB()
ros = RandomOverSampler(random_state=42)

X_train_ros, y_train_ros= ros.fit_resample(xtrain_tfidf, y_train)
text_clf = text_clf.fit(X_train_ros, y_train_ros)
predictions = text_clf.predict(xvalid_tfidf)

In [57]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("random oversampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

random oversampling
Accuracy: 0.8718428437792329
Precision: 0.7197518097207859
recall: 0.9957081545064378
f1_score: 0.8355342136854741


In [58]:
#random undersampling
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus= rus.fit_resample(xtrain_tfidf, y_train)
text_clf = MultinomialNB()
text_clf = text_clf.fit(X_train_rus, y_train_rus)
predictions = text_clf.predict(xvalid_tfidf)

In [59]:
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("random undersampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

random undersampling
Accuracy: 0.8709073900841908
Precision: 0.718717683557394
recall: 0.994277539341917
f1_score: 0.8343337334933973


# SVM classifier

In [49]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, random_state=42)),
])

text_clf_svm = text_clf_svm.fit(X_train, y_train)

In [50]:
predictions = text_clf_svm.predict(X_test)

### Evaluating the SVM classifier

In [51]:
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

Accuracy: 0.8699719363891487
Precision: 0.7158974358974359
recall: 0.9985693848354793
f1_score: 0.8339307048984469


In [80]:
#random oversampling
text_clf_svm = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42)

ros = RandomOverSampler(random_state=42)

X_train_ros, y_train_ros= ros.fit_resample(xtrain_tfidf, y_train)
text_clf_svm = text_clf_svm.fit(X_train_ros, y_train_ros)
predictions = text_clf_svm.predict(xvalid_tfidf)

In [81]:
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("oversampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

oversampling
Accuracy: 0.8685687558465855
Precision: 0.713265306122449
recall: 1.0
f1_score: 0.8326384752829065


In [82]:
#random undersampling
text_clf_svm = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42)

rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus= rus.fit_resample(xtrain_tfidf, y_train)
text_clf_svm = text_clf_svm.fit(X_train_rus, y_train_rus)
predictions = text_clf_svm.predict(xvalid_tfidf)

In [83]:
conf_matrix = confusion_matrix(y_test, predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("undersampling")
print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

undersampling
Accuracy: 0.8685687558465855
Precision: 0.713265306122449
recall: 1.0
f1_score: 0.8326384752829065


# Logistic regression

In [69]:
from sklearn.linear_model import LogisticRegression
# Train model
clf_lg = LogisticRegression()  
clf_lg = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf',LogisticRegression()),
])
clf_lg = clf_lg.fit(X_train, y_train) 

In [70]:
lg_predictions = clf_lg.predict(X_test)

### Evaluating logistic regression classifier

In [71]:
conf_matrix = confusion_matrix(y_test, lg_predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

Accuracy: 0.9106641721234799
Precision: 0.7995283018867925
recall: 0.9699570815450643
f1_score: 0.8765352294764058


In [74]:
#random oversampling
clf_lg = LogisticRegression()  

ros = RandomOverSampler(random_state=42)

X_train_ros, y_train_ros= ros.fit_resample(xtrain_tfidf, y_train)
clf_lg = clf_lg.fit(X_train_ros, y_train_ros)
lg_predictions = clf_lg.predict(xvalid_tfidf)

In [75]:
conf_matrix = confusion_matrix(y_test, lg_predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

Accuracy: 0.8947614593077643
Precision: 0.7615894039735099
recall: 0.9871244635193133
f1_score: 0.8598130841121495


In [76]:
#random undersampling
clf_lg = LogisticRegression()  

ros = RandomUnderSampler(random_state=42)

X_train_rus, y_train_rus= rus.fit_resample(xtrain_tfidf, y_train)
clf_lg = clf_lg.fit(X_train_rus, y_train_rus)
lg_predictions = clf_lg.predict(xvalid_tfidf)

In [77]:
conf_matrix = confusion_matrix(y_test, lg_predictions)
tn = conf_matrix[0][0]
fn = conf_matrix[1][0]
tp = conf_matrix[1][1]
fp = conf_matrix[0][1]

accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2*(precision*recall)/(precision + recall)

print("Accuracy:",accuracy)
print("Precision:",precision)
print("recall:",recall)
print("f1_score:",f1_score)

Accuracy: 0.8802619270346118
Precision: 0.7348886532343585
recall: 0.9914163090128756
f1_score: 0.8440925700365408
